In [ ]:
import os
import re
import json
import chardet
import scr.public_function as pf

# NOTE 中间的所有文件都是UTF8
encoding = 'cp932'
output_encoding = 'utf16'

pf._init_()
name = '[171231] [Miel] 文芸彼女と僕二人だけの愛の巣だった部室にヤリチンが入部してきた！ DL版 (files)'

In [ ]:
# FIXME 从游戏原文本文件提取日文


def get_scenario_from_origin(data: list) -> list:
    text_all = []
    cnt = 0
    # START FIXME
    for line in data:
        if pf.has_jp(line) and line[0] not in "@*;\t /S":
            text_all.append(line)
        cnt+=1
    # END
    return text_all

pf.extract_jp(get_scenario_from_origin, encoding)


In [ ]:
# FIXME 创建翻译字典
pf.create_dict(lambda x:x[:-1])

In [ ]:
# 检查字典key数量是否正确
pf.check_dict()

In [ ]:
# 把字典翻译


# print(delete_f(''))
pf.translate(delete_func=None)
# print(pf._translate('「沙希はぁ……みなさまのぉ……オナペット奴隷ですぅ[haret]　い、いっしょうけんめい……恥ずかしいことぉ……エッ　チなことぉ……たくさんしますからぁ……[haret]」[sv][plc][botan]', delete_func=None))

In [ ]:
# 检查是否还有没翻译的语句
pf.check_dict_untranslated()

In [ ]:
# 构建对比
pf.create_contrast()

In [ ]:

def find_all(data:list, failed_text:list, jp_chs:dict):
    record = 0
    cnt = 0                         # 当前行
    start = False
    for line in data:
        #START FIXME 替换目标文本
        if pf.has_jp(line) and line[0] not in "@*;\t /":
            key = line[:-1]
            if key in jp_chs and jp_chs[key]:
                record += 1
                data[cnt] = data[cnt].replace(key, jp_chs[key])
            else:
                failed_text.append(key+'\n')
        cnt += 1
        # END 
    return record

pf.replace_all(find_all, encoding, output_encoding)

In [ ]:
pf.YU_RIS.extract_ybn()